In [1]:
import random

from datasets import load_dataset
import numpy as np
import torch
import transformers

from life_after_bert import MCDataset, evaluate_encoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "roberta-large"
task_type = "oLMpics MLM"
num_choices = 2

In [3]:
model = transformers.AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [4]:
dataset = load_dataset("KevinZ/Ettinger", "CPRAG", use_auth_token=True)["test"]

def preprocess_CPRAG(example):
    question = f'{example["context_s1"]} {example["context_s2"]} [MASK]'
    choices = [example[key] for key in ["expected", "within_category", "between_category"]]
    random.shuffle(choices)
    answer_id = choices.index(example["expected"])

    return {
        "questions": question,
        "choices": choices,
        "answer_ids": answer_id,
    }

dataset = dataset.map(preprocess_CPRAG)
dataset = MCDataset(dataset["questions"], dataset["choices"], dataset["answer_ids"], num_choices, task_type, tokenizer, max_length=36)

2022-04-01 12:49:35 | WARNING | datasets.builder | Reusing dataset new_dataset (/home/kzhao/.cache/huggingface/datasets/KevinZ___new_dataset/CPRAG/1.0.0/2a3d83194518f1e650e3ae8e54ce0d0ee0bfa69293ce1fcb4b79ec55621d035c)


  0%|          | 0/1 [00:00<?, ?it/s]

2022-04-01 12:49:35 | WARNING | datasets.arrow_dataset | Loading cached processed dataset at /home/kzhao/.cache/huggingface/datasets/KevinZ___new_dataset/CPRAG/1.0.0/2a3d83194518f1e650e3ae8e54ce0d0ee0bfa69293ce1fcb4b79ec55621d035c/cache-b72199c247e9361c.arrow
2022-04-01 12:49:35 | WARNING | data.py | Truncated 13 tokens from answer choices.


In [5]:
accuracy, (all_answers, all_preds) = evaluate_encoder(model, tokenizer, dataset, device=device)
accuracy

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

0.8529411764705882